# Feature Engineering

### Get data from DB

In [3]:
import pandas as pd
import numpy as np
import sqlite3

pd.set_option("display.max_columns", None)

conn = sqlite3.connect("../data/ufc.db", detect_types=sqlite3.PARSE_DECLTYPES)
query = """
        SELECT
            a.*,
            b.HEIGHT_INCHES AS RED_HEIGHT_INCHES,
            b.REACH_INCHES AS RED_REACH_INCHES,
            b.FIGHTING_STANCE AS RED_FIGHTING_STANCE,
            b.DATE_OF_BIRTH AS RED_DATE_OF_BIRTH,
            c.HEIGHT_INCHES AS BLUE_HEIGHT_INCHES,
            c.REACH_INCHES AS BLUE_REACH_INCHES,
            c.FIGHTING_STANCE AS BLUE_FIGHTING_STANCE,
            c.DATE_OF_BIRTH AS BLUE_DATE_OF_BIRTH
        FROM
            BOUTS_OVERALL a
        LEFT JOIN
            FIGHTERS b
        ON
            a.RED_FIGHTER_ID = b.FIGHTER_ID
        LEFT JOIN
            FIGHTERS c
        ON
            a.BLUE_FIGHTER_ID = c.FIGHTER_ID
        """

df = pd.read_sql_query(query, conn)
conn.close()
df = df.iloc[:7289]

In [4]:
df

,BOUT_ID,EVENT_ID,EVENT_NAME,DATE,LOCATION,RED_FIGHTER_ID,BLUE_FIGHTER_ID,RED_FIGHTER_NAME,BLUE_FIGHTER_NAME,RED_OUTCOME,BLUE_OUTCOME,BOUT_TYPE,OUTCOME_METHOD,OUTCOME_METHOD_DETAILS,END_ROUND,END_ROUND_TIME_MINUTES,BOUT_TIME_FORMAT,TOTAL_TIME_MINUTES,RED_KNOCKDOWNS,BLUE_KNOCKDOWNS,RED_TOTAL_STRIKES_LANDED,RED_TOTAL_STRIKES_ATTEMPTED,BLUE_TOTAL_STRIKES_LANDED,BLUE_TOTAL_STRIKES_ATTEMPTED,RED_TAKEDOWNS_LANDED,RED_TAKEDOWNS_ATTEMPTED,BLUE_TAKEDOWNS_LANDED,BLUE_TAKEDOWNS_ATTEMPTED,RED_SUBMISSION_ATTEMPTS,BLUE_SUBMISSION_ATTEMPTS,RED_REVERSALS,BLUE_REVERSALS,RED_CONTROL_TIME_MINUTES,BLUE_CONTROL_TIME_MINUTES,RED_SIGNIFICANT_STRIKES_LANDED,RED_SIGNIFICANT_STRIKES_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_LANDED,BLUE_SIGNIFICANT_STRIKES_ATTEMPTED,RED_SIGNIFICANT_STRIKES_HEAD_LANDED,RED_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_HEAD_LANDED,BLUE_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED,RED_SIGNIFICANT_STRIKES_BODY_LANDED,RED_SIGNIFICANT_STRIKES_BODY_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_BODY_LANDED,BLUE_SIGNIFICANT_STRIKES_BODY_ATTEMPTED,RED_SIGNIFICANT_STRIKES_LEG_LANDED,RED_SIGNIFICANT_STRIKES_LEG_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_LEG_LANDED,BLUE_SIGNIFICANT_STRIKES_LEG_ATTEMPTED,RED_SIGNIFICANT_STRIKES_DISTANCE_LANDED,RED_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_DISTANCE_LANDED,BLUE_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED,RED_SIGNIFICANT_STRIKES_CLINCH_LANDED,RED_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_CLINCH_LANDED,BLUE_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED,RED_SIGNIFICANT_STRIKES_GROUND_LANDED,RED_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_GROUND_LANDED,BLUE_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED,RED_HEIGHT_INCHES,RED_REACH_INCHES,RED_FIGHTING_STANCE,RED_DATE_OF_BIRTH,BLUE_HEIGHT_INCHES,BLUE_REACH_INCHES,BLUE_FIGHTING_STANCE,BLUE_DATE_OF_BIRTH
0,567a09fd200cfa05,6420efac0578988b,UFC 1: The Beginning,1993-11-12,"Denver, Colorado, USA",279093302a6f44b3,96eff1a628adcc7f,Gerard Gordeau,Teila Tuli,W,L,Open Weight Bout,KO/TKO,Kick to Head On Ground,1,0.433333,No Time Limit,0.433333,0.0,0.0,3.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,3.0,5.0,0.0,1.0,3.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,77.0,NaN,Orthodox,1959-03-30,72.0,NaN,Orthodox,1969-06-14
1,2d2bbc86e941e05c,6420efac0578988b,UFC 1: The Beginning,1993-11-12,"Denver, Colorado, USA",598a58db87b890ee,d3711d3784b76255,Kevin Rosier,Zane Frazier,W,L,Open Weight Bout,KO/TKO,to \n Corner Stoppage,1,4.333333,No Time Limit,4.333333,2.0,0.0,38.0,53.0,13.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,15.0,27.0,12.0,28.0,12.0,23.0,7.0,19.0,3.0,4.0,3.0,6.0,0.0,0.0,2.0,3.0,4.0,10.0,0.0,7.0,4.0,9.0,10.0,19.0,7.0,8.0,2.0,2.0,76.0,NaN,Orthodox,None,77.0,NaN,Orthodox,None
2,cecdc0da584274b9,6420efac0578988b,UFC 1: The Beginning,1993-11-12,"Denver, Colorado, USA",429e7d3725852ce9,a5c53b3ddb31cc7d,Royce Gracie,Art Jimmerson,W,L,Open Weight Bout,Submission,Other \n Position - Mount,1,2.300000,No Time Limit,2.300000,0.0,0.0,4.0,7.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0,NaN,Southpaw,1966-12-12,73.0,NaN,Orthodox,1963-08-04
3,46acd54cc0c905fb,6420efac0578988b,UFC 1: The Beginning,1993-11-12,"Denver, Colorado, USA",63b65af1c5cb02cb,46c8ec317aff28ac,Ken Shamrock,Patrick Smith,W,L,Open Weight Bout,Submission,Heel Hook From Guard,1,1.816667,No Time Limit,1.816667,0.0,0.0,4.0,4.0,16.0,20.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,4.0,8.0,1.0,1.0,1.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,6.0,73.0,72.0,Orthodox,1964-02-11,74.0,NaN,Orthodox,1963-08-28
4,ac7ca2ec38b96c1a,6420efac0578988b,UFC 1: The Beginning,1993-11-12,"Denver, Colorado, USA",279093302a6f44b3,598a58db87b890ee,Gerard Gordeau,Kevin Rosier,W,L,Open Weight Bout,KO/TKO,to \n Corner Stoppage,1,0.983333,No Time Limit,0.983333,1.0,0.0,11.0,17.0,0.0,3.0,0.0,

### Create a stacked dataset so that features can be engineered by fighter

In [2]:
def get_weight_class(bout_type):
    if "Women's Strawweight" in bout_type:
        return "Women's Strawweight"
    elif "Women's Flyweight" in bout_type:
        return "Women's Flyweight"
    elif "Women's Bantamweight" in bout_type:
        return "Women's Bantamweight"
    elif "Women's Featherweight" in bout_type:
        return "Women's Featherweight"
    elif "Light Heavyweight" in bout_type:
        return "Light Heavyweight"
    elif "Heavyweight" in bout_type:
        return "Heavyweight"
    elif "Middleweight" in bout_type:
        return "Middleweight"
    elif "Welterweight" in bout_type:
        return "Welterweight"
    elif "Lightweight" in bout_type:
        return "Lightweight"
    elif "Featherweight" in bout_type:
        return "Featherweight"
    elif "Bantamweight" in bout_type:
        return "Bantamweight"
    elif "Flyweight" in bout_type:
        return "Flyweight"
    elif "Catch Weight" in bout_type:
        return "Catch Weight"
    elif "Open Weight" in bout_type:
        return "Open Weight"
    
def is_female(bout_type):
    if "Women's" in bout_type:
        return 1
    return 0

def is_title_bout(bout_type):
    if any(x in bout_type for x in ["Tournament Title Bout", "Ultimate Fighter", "TUF"]):
        return 0
    elif "Title Bout" in bout_type:
        return 1
    return 0

In [3]:
temp = df.copy().rename_axis("TEMP").reset_index()
common = [
    "TEMP",
    "BOUT_ID",
    "DATE",
    # "LOCATION",
    "BOUT_TYPE",
    "OUTCOME_METHOD",
    "END_ROUND",
    "BOUT_TIME_FORMAT",
    "TOTAL_TIME_MINUTES",
]
red_cols = [col for col in temp.columns if col.startswith("RED_")]
blue_cols = [col for col in temp.columns if col.startswith("BLUE_")]

reds = temp[common + red_cols + blue_cols]
reds = reds.rename(
    columns=lambda x: x.replace("RED_", "f_")
    if x.startswith("RED_")
    else x.replace("BLUE_", "o_")
    if x.startswith("BLUE_")
    else x
)
reds["TEMP2"] = 1
blues = temp[common + blue_cols + red_cols]
blues = blues.rename(
    columns=lambda x: x.replace("BLUE_", "f_")
    if x.startswith("BLUE_")
    else x.replace("RED_", "o_")
    if x.startswith("RED_")
    else x
)
blues["TEMP2"] = 2

feature_df = (
    pd.concat([reds, blues])
    .sort_values(["TEMP", "TEMP2"])
    .drop(["TEMP", "TEMP2", "f_FIGHTER_NAME", "o_FIGHTER_NAME", "o_DATE_OF_BIRTH", "o_FIGHTER_ID"], axis=1)
)
feature_df["DATE"] = pd.to_datetime(feature_df["DATE"])
feature_df["f_DATE_OF_BIRTH"] = pd.to_datetime(feature_df["f_DATE_OF_BIRTH"])
feature_df = feature_df.drop("o_OUTCOME", axis=1)

# Bout type feature engineering
feature_df["BOUT_WEIGHT_CLASS"] = feature_df["BOUT_TYPE"].apply(lambda x: get_weight_class(x))
feature_df["BOUT_IS_FEMALE"] = feature_df["BOUT_TYPE"].apply(lambda x: is_female(x))
feature_df["BOUT_IS_TITLE_BOUT"] = feature_df["BOUT_TYPE"].apply(lambda x: is_title_bout(x))
feature_df["f_WIN_TITLE_BOUT"] = np.where((feature_df["f_OUTCOME"] == "W") & (feature_df["BOUT_IS_TITLE_BOUT"] == 1), 1, 0)
feature_df = feature_df.drop("BOUT_TYPE", axis=1)

# Outcome and method feature engineering
feature_df["f_WIN"] = np.where(feature_df["f_OUTCOME"] == "W", 1, 0)
feature_df["f_LOSS"] = np.where(feature_df["f_OUTCOME"] == "L", 1, 0)
feature_df["f_DRAW"] = np.where(feature_df["f_OUTCOME"] == "D", 1, 0)
feature_df["f_NO_CONTEST"] = np.where(feature_df["f_OUTCOME"] == "NC", 1, 0)
feature_df["f_WIN_BY_KO/TKO"] = np.where((feature_df["f_OUTCOME"] == "W") & ((feature_df["OUTCOME_METHOD"] == "KO/TKO") | (feature_df["OUTCOME_METHOD"] == "TKO - Doctor's Stoppage")), 1, 0)
feature_df["f_WIN_BY_SUBMISSION"] = np.where((feature_df["f_OUTCOME"] == "W") & (feature_df["OUTCOME_METHOD"] == "Submission"), 1, 0)
feature_df["f_WIN_BY_DECISION"] = np.where((feature_df["f_OUTCOME"] == "W") & (feature_df["OUTCOME_METHOD"].str.contains("Decision")), 1, 0)
feature_df["f_LOSS_BY_KO/TKO"] = np.where((feature_df["f_OUTCOME"] == "L") & ((feature_df["OUTCOME_METHOD"] == "KO/TKO") | (feature_df["OUTCOME_METHOD"] == "TKO - Doctor's Stoppage")), 1, 0)
feature_df["f_LOSS_BY_SUBMISSION"] = np.where((feature_df["f_OUTCOME"] == "L") & (feature_df["OUTCOME_METHOD"] == "Submission"), 1, 0)
feature_df["f_LOSS_BY_DECISION"] = np.where((feature_df["f_OUTCOME"] == "L") & (feature_df["OUTCOME_METHOD"].str.contains("Decision")), 1, 0)
feature_df = feature_df.drop(["f_OUTCOME", "OUTCOME_METHOD"], axis=1)

# End round and time format stuff
feature_df["f_WIN_FIRST_ROUND_FINISH"] = feature_df.apply(lambda x: 1 if (x["f_WIN"] == 1 and x["END_ROUND"] == 1) else 0, axis=1)
feature_df["BOUT_IS_FIVE_ROUNDS"] = feature_df["BOUT_TIME_FORMAT"].apply(lambda x: 1 if x == "5 Rnd (5-5-5-5-5)" else 0)
feature_df["f_WIN_FIVE_ROUND_BOUT"] = np.where((feature_df["f_WIN"] == 1) & (feature_df["BOUT_IS_FIVE_ROUNDS"] == 1), 1, 0)
feature_df = feature_df.drop(["END_ROUND", "BOUT_TIME_FORMAT"], axis=1)

# feature_df

In [4]:
cumulative_features = pd.DataFrame()
cumulative_features["f_TOTAL_TIME_MINUTES_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["TOTAL_TIME_MINUTES"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_KNOCKDOWNS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_KNOCKDOWNS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_TOTAL_STRIKES_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_TOTAL_STRIKES_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_TOTAL_STRIKES_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_TAKEDOWNS_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_TAKEDOWNS_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_TAKEDOWNS_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_TAKEDOWNS_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SUBMISSION_ATTEMPTS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SUBMISSION_ATTEMPTS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_REVERSALS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_REVERSALS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_CONTROL_TIME_MINUTES_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_CONTROL_TIME_MINUTES"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_HEAD_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_BODY_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_BODY_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_LEG_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_LEG_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_CLINCH_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_GROUND_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_KNOCKDOWNS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_KNOCKDOWNS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_TOTAL_STRIKES_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_TOTAL_STRIKES_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_TOTAL_STRIKES_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_TAKEDOWNS_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_TAKEDOWNS_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_TAKEDOWNS_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_TAKEDOWNS_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SUBMISSION_ATTEMPTS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SUBMISSION_ATTEMPTS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_REVERSALS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_REVERSALS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_CONTROL_TIME_MINUTES_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_CONTROL_TIME_MINUTES"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_HEAD_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_BODY_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_LEG_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_DISTANCE_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_CLINCH_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_GROUND_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))

cumulative_features["f_TITLE_BOUTS_FOUGHT"] = feature_df.groupby("f_FIGHTER_ID")["BOUT_IS_TITLE_BOUT"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_WINS_TITLE_BOUTS"] = feature_df.groupby("f_FIGHTER_ID")["f_WIN_TITLE_BOUT"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_FIVE_ROUND_BOUTS_FOUGHT"] = feature_df.groupby("f_FIGHTER_ID")["BOUT_IS_FIVE_ROUNDS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_WINS_FIVE_ROUND_BOUTS"] = feature_df.groupby("f_FIGHTER_ID")["f_WIN_FIVE_ROUND_BOUT"].transform(lambda x: x.cumsum().shift(1))

cumulative_features["f_WINS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_WIN"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_LOSSES_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_LOSS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_DRAWS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_DRAW"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_NO_CONTESTS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_NO_CONTEST"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_TOTAL_BOUTS_FOUGHT"] = cumulative_features["f_WINS_CUMULATIVE"] + cumulative_features["f_LOSSES_CUMULATIVE"] + cumulative_features["f_DRAWS_CUMULATIVE"] + cumulative_features["f_NO_CONTESTS_CUMULATIVE"]
cumulative_features["f_WINS_BY_KO/TKO_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_WIN_BY_KO/TKO"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_WINS_BY_SUBMISSION_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_WIN_BY_SUBMISSION"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_WINS_BY_DECISION_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_WIN_BY_DECISION"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_WINS_FIRST_ROUND_FINISH_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_WIN_FIRST_ROUND_FINISH"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_LOSSES_BY_KO/TKO_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_LOSS_BY_KO/TKO"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_LOSSES_BY_SUBMISSION_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_LOSS_BY_SUBMISSION"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_LOSSES_BY_DECISION_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_LOSS_BY_DECISION"].transform(lambda x: x.cumsum().shift(1))

# cumulative_features

In [5]:
cumulative_features2 = cumulative_features.copy()
cumulative_features2["f_KNOCKDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["f_KNOCKDOWNS_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_KNOCKDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["o_KNOCKDOWNS_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_KNOCKDOWNS_LANDED/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE"] = cumulative_features2["f_KNOCKDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE"] - cumulative_features2["f_KNOCKDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE"]
cumulative_features2["f_KNOCKDOWNS_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_KNOCKDOWNS_CUMULATIVE"] - cumulative_features2["o_KNOCKDOWNS_CUMULATIVE"]

cumulative_features2["f_TOTAL_STRIKES_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_TOTAL_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_TOTAL_STRIKES_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_TOTAL_STRIKES_LANDED_CUMULATIVE"]) / cumulative_features2["o_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_TOTAL_STRIKES_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_TOTAL_STRIKES_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_TOTAL_STRIKES_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_TOTAL_STRIKES_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_TOTAL_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_TOTAL_STRIKES_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_TOTAL_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_TOTAL_STRIKES_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_TOTAL_STRIKES_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_TOTAL_STRIKES_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_TOTAL_STRIKES_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_TOTAL_STRIKES_LANDED_CUMULATIVE"] - cumulative_features2["o_TOTAL_STRIKES_LANDED_CUMULATIVE"]

cumulative_features2["f_TAKEDOWNS_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_TAKEDOWNS_LANDED_CUMULATIVE"] / cumulative_features2["f_TAKEDOWNS_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_TAKEDOWNS_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_TAKEDOWNS_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_TAKEDOWNS_LANDED_CUMULATIVE"]) / cumulative_features2["o_TAKEDOWNS_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_TAKEDOWNS_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_TAKEDOWNS_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_TAKEDOWNS_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_TAKEDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["f_TAKEDOWNS_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_TAKEDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["o_TAKEDOWNS_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_TAKEDOWNS_LANDED/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE"] = cumulative_features2["f_TAKEDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE"] - cumulative_features2["f_TAKEDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE"]
cumulative_features2["f_TAKEDOWNS_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_TAKEDOWNS_LANDED_CUMULATIVE"] - cumulative_features2["o_TAKEDOWNS_LANDED_CUMULATIVE"]

cumulative_features2["f_SUBMISSION_ATTEMPTS_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_WINS_BY_SUBMISSION_CUMULATIVE"] / (1 + cumulative_features2["f_SUBMISSION_ATTEMPTS_CUMULATIVE"])
cumulative_features2["f_SUBMISSION_ATTEMPTS_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SUBMISSION_ATTEMPTS_CUMULATIVE"] - cumulative_features2["f_LOSSES_BY_SUBMISSION_CUMULATIVE"]) / (1 + cumulative_features2["o_SUBMISSION_ATTEMPTS_CUMULATIVE"])
cumulative_features2["f_SUBMISSION_ATTEMPTS_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SUBMISSION_ATTEMPTS_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SUBMISSION_ATTEMPTS_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SUBMISSION_ATTEMPTS_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["f_SUBMISSION_ATTEMPTS_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_SUBMISSION_ATTEMPTS_ABSORBED_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["o_SUBMISSION_ATTEMPTS_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_SUBMISSION_ATTEMPTS/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE"] = cumulative_features2["f_SUBMISSION_ATTEMPTS_PER_15_MINUTES_CUMULATIVE"] - cumulative_features2["f_SUBMISSION_ATTEMPTS_ABSORBED_PER_15_MINUTES_CUMULATIVE"]
cumulative_features2["f_SUBMISSION_ATTEMPTS/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SUBMISSION_ATTEMPTS_CUMULATIVE"] - cumulative_features2["o_SUBMISSION_ATTEMPTS_CUMULATIVE"]

cumulative_features2["f_REVERSALS_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_REVERSALS_CUMULATIVE"] / (1 + cumulative_features2["o_TAKEDOWNS_LANDED_CUMULATIVE"])
cumulative_features2["f_REVERSALS_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["f_TAKEDOWNS_LANDED_CUMULATIVE"] - cumulative_features2["o_REVERSALS_CUMULATIVE"]) / (1 + cumulative_features2["f_TAKEDOWNS_LANDED_CUMULATIVE"])
cumulative_features2["f_REVERSALS_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_REVERSALS_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_REVERSALS_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_REVERSALS_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["f_REVERSALS_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_REVERSALS_ABSORBED_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["o_REVERSALS_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_REVERSALS/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE"] = cumulative_features2["f_REVERSALS_PER_15_MINUTES_CUMULATIVE"] - cumulative_features2["f_REVERSALS_ABSORBED_PER_15_MINUTES_CUMULATIVE"]
cumulative_features2["f_REVERSALS/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_REVERSALS_CUMULATIVE"] - cumulative_features2["o_REVERSALS_CUMULATIVE"]

cumulative_features2["f_CONTROL_TIME_PROPORTION_CUMULATIVE"] = cumulative_features2["f_CONTROL_TIME_MINUTES_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_CONTROL_TIME_GIVEN_UP_PROPORTION_CUMULATIVE"] = cumulative_features2["o_CONTROL_TIME_MINUTES_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_CONTROL_TIME/CONTROL_TIME_GIVEN_UP_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_CONTROL_TIME_PROPORTION_CUMULATIVE"] - cumulative_features2["f_CONTROL_TIME_GIVEN_UP_PROPORTION_CUMULATIVE"]
cumulative_features2["f_CONTROL_TIME/CONTROL_TIME_GIVEN_UP_DIFF_CUMULATIVE"] = cumulative_features2["f_CONTROL_TIME_MINUTES_CUMULATIVE"] - cumulative_features2["o_CONTROL_TIME_MINUTES_CUMULATIVE"]

cumulative_features2 = cumulative_features2.copy()

cumulative_features2["f_SIGNIFICANT_STRIKES_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_OUT_OF_TOTAL_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_ABSORBED_OUT_OF_TOTAL_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["o_TOTAL_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED/ABSORBED_OUT_OF_TOTAL_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_OUT_OF_TOTAL_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_ABSORBED_OUT_OF_TOTAL_PROPORTION_CUMULATIVE"]

cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] / cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED/ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"]

cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] / cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED/ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"]

cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] / cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED/ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"]

cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] / cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED/ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"]

cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] / cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED/ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"]

cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] / cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED/ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"]

# cumulative_features2

In [6]:
cumulative_features3 = cumulative_features2.copy()

cumulative_features3["f_FIGHTER_ID"] = feature_df["f_FIGHTER_ID"]
cumulative_features3["TOTAL_TIME_MINUTES_WIN"] = np.where(feature_df["f_WIN"] == 1, feature_df["TOTAL_TIME_MINUTES"], 0)
cumulative_features3["f_TOTAL_TIME_MINUTES_WIN_CUMULATIVE"] = cumulative_features3.groupby("f_FIGHTER_ID")["TOTAL_TIME_MINUTES_WIN"].transform(lambda x: x.cumsum().shift(1))
cumulative_features3["f_AVERAGE_TIME_TO_WIN_CUMULATIVE"] = cumulative_features3["f_TOTAL_TIME_MINUTES_WIN_CUMULATIVE"] / cumulative_features3["f_WINS_CUMULATIVE"]
cumulative_features3["TOTAL_TIME_MINUTES_LOSS"] = np.where(feature_df["f_LOSS"] == 1, feature_df["TOTAL_TIME_MINUTES"], 0)
cumulative_features3["f_TOTAL_TIME_MINUTES_LOSS_CUMULATIVE"] = cumulative_features3.groupby("f_FIGHTER_ID")["TOTAL_TIME_MINUTES_LOSS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features3["f_AVERAGE_TIME_TO_LOSE_CUMULATIVE"] = cumulative_features3["f_TOTAL_TIME_MINUTES_LOSS_CUMULATIVE"] / cumulative_features3["f_LOSSES_CUMULATIVE"]
cumulative_features3["f_AVERAGE_TIME_TO_WIN/LOSE_DIFF_CUMULATIVE"] = cumulative_features3["f_AVERAGE_TIME_TO_WIN_CUMULATIVE"] - cumulative_features3["f_AVERAGE_TIME_TO_LOSE_CUMULATIVE"]
cumulative_features3 = cumulative_features3.drop(["f_FIGHTER_ID", "TOTAL_TIME_MINUTES_WIN", "TOTAL_TIME_MINUTES_LOSS", "f_TOTAL_TIME_MINUTES_WIN_CUMULATIVE", "f_TOTAL_TIME_MINUTES_LOSS_CUMULATIVE"], axis=1)

cumulative_features3["f_WIN_PROPORTION_CUMULATIVE"] = cumulative_features3["f_WINS_CUMULATIVE"] / cumulative_features3["f_TOTAL_BOUTS_FOUGHT"]
cumulative_features3["f_LOSS_PROPORTION_CUMULATIVE"] = cumulative_features3["f_LOSSES_CUMULATIVE"] / cumulative_features3["f_TOTAL_BOUTS_FOUGHT"]
cumulative_features3["f_WIN/LOSS_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features3["f_WIN_PROPORTION_CUMULATIVE"] - cumulative_features3["f_LOSS_PROPORTION_CUMULATIVE"]
cumulative_features3["f_WIN/LOSS_DIFF_CUMULATIVE"] = cumulative_features3["f_WINS_CUMULATIVE"] - cumulative_features3["f_LOSSES_CUMULATIVE"]

cumulative_features3["f_WIN_BY_KO/TKO_OUT_OF_WINS_PROPORTION_CUMULATIVE"] = cumulative_features3["f_WINS_BY_KO/TKO_CUMULATIVE"] / cumulative_features3["f_WINS_CUMULATIVE"]
cumulative_features3["f_LOSS_BY_KO/TKO_OUT_OF_LOSSES_PROPORTION_CUMULATIVE"] = cumulative_features3["f_LOSSES_BY_KO/TKO_CUMULATIVE"] / cumulative_features3["f_LOSSES_CUMULATIVE"]
cumulative_features3["f_WIN/LOSS_BY_KO/TKO_OUT_OF_WINS/LOSSES_PROPORION_DIFF_CUMULATIVE"] = cumulative_features3["f_WIN_BY_KO/TKO_OUT_OF_WINS_PROPORTION_CUMULATIVE"] - cumulative_features3["f_LOSS_BY_KO/TKO_OUT_OF_LOSSES_PROPORTION_CUMULATIVE"]
cumulative_features3["f_WIN/LOSS_BY_KO/TKO_DIFF_CUMULATIVE"] = cumulative_features3["f_WINS_BY_KO/TKO_CUMULATIVE"] - cumulative_features3["f_LOSSES_BY_KO/TKO_CUMULATIVE"]
cumulative_features3["f_WIN_BY_SUBMISSION_OUT_OF_WINS_PROPORTION_CUMULATIVE"] = cumulative_features3["f_WINS_BY_SUBMISSION_CUMULATIVE"] / cumulative_features3["f_WINS_CUMULATIVE"]
cumulative_features3["f_LOSS_BY_SUBMISSION_OUT_OF_LOSSES_PROPORTION_CUMULATIVE"] = cumulative_features3["f_LOSSES_BY_SUBMISSION_CUMULATIVE"] / cumulative_features3["f_LOSSES_CUMULATIVE"]
cumulative_features3["f_WIN/LOSS_BY_SUBMISSION_OUT_OF_WINS/LOSSES_PROPORION_DIFF_CUMULATIVE"] = cumulative_features3["f_WIN_BY_SUBMISSION_OUT_OF_WINS_PROPORTION_CUMULATIVE"] - cumulative_features3["f_LOSS_BY_SUBMISSION_OUT_OF_LOSSES_PROPORTION_CUMULATIVE"]
cumulative_features3["f_WIN/LOSS_BY_SUBMISSION_DIFF_CUMULATIVE"] = cumulative_features3["f_WINS_BY_SUBMISSION_CUMULATIVE"] - cumulative_features3["f_LOSSES_BY_SUBMISSION_CUMULATIVE"]
cumulative_features3["f_WIN_BY_DECISION_OUT_OF_WINS_PROPORTION_CUMULATIVE"] = cumulative_features3["f_WINS_BY_DECISION_CUMULATIVE"] / cumulative_features3["f_WINS_CUMULATIVE"]
cumulative_features3["f_LOSS_BY_DECISION_OUT_OF_LOSSES_PROPORTION_CUMULATIVE"] = cumulative_features3["f_LOSSES_BY_DECISION_CUMULATIVE"] / cumulative_features3["f_LOSSES_CUMULATIVE"]
cumulative_features3["f_WIN/LOSS_BY_DECISION_OUT_OF_WINS/LOSSES_PROPORION_DIFF_CUMULATIVE"] = cumulative_features3["f_WIN_BY_DECISION_OUT_OF_WINS_PROPORTION_CUMULATIVE"] - cumulative_features3["f_LOSS_BY_DECISION_OUT_OF_LOSSES_PROPORTION_CUMULATIVE"]
cumulative_features3["f_WIN/LOSS_BY_DECISION_DIFF_CUMULATIVE"] = cumulative_features3["f_WINS_BY_DECISION_CUMULATIVE"] - cumulative_features3["f_LOSSES_BY_DECISION_CUMULATIVE"]

cumulative_features3["f_WIN_BY_FINISH_OUT_OF_WINS_PROPORTION_CUMULATIVE"] = (cumulative_features3["f_WINS_BY_KO/TKO_CUMULATIVE"] + cumulative_features3["f_WINS_BY_SUBMISSION_CUMULATIVE"]) / cumulative_features3["f_WINS_CUMULATIVE"]
cumulative_features3["f_LOSS_BY_FINISH_OUT_OF_LOSSES_PROPORTION_CUMULATIVE"] = (cumulative_features3["f_LOSSES_BY_KO/TKO_CUMULATIVE"] + cumulative_features3["f_LOSSES_BY_SUBMISSION_CUMULATIVE"]) / cumulative_features3["f_LOSSES_CUMULATIVE"]
cumulative_features3["f_WIN/LOSS_BY_FINISH_OUT_OF_WINS/LOSSES_PROPORION_DIFF_CUMULATIVE"] = cumulative_features3["f_WIN_BY_FINISH_OUT_OF_WINS_PROPORTION_CUMULATIVE"] - cumulative_features3["f_LOSS_BY_FINISH_OUT_OF_LOSSES_PROPORTION_CUMULATIVE"]
cumulative_features3["f_WIN/LOSS_BY_FINISH_DIFF_CUMULATIVE"] = (cumulative_features3["f_WINS_BY_KO/TKO_CUMULATIVE"] + cumulative_features3["f_WINS_BY_SUBMISSION_CUMULATIVE"]) - (cumulative_features3["f_LOSSES_BY_KO/TKO_CUMULATIVE"] + cumulative_features3["f_LOSSES_BY_SUBMISSION_CUMULATIVE"])

cumulative_features3["f_SIGNIFICANT_STRIKES_GROUND_LANDED_PER_MINUTE_OF_CONTROL_CUMULATIVE"] = cumulative_features3["f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] / (1 + cumulative_features3["f_CONTROL_TIME_MINUTES_CUMULATIVE"])
cumulative_features3["f_SIGNIFICANT_STRIKES_GROUND_ABSORBED_PER_MINUTE_OF_CONTROL_CUMULATIVE"] = cumulative_features3["o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] / (1 + cumulative_features3["o_CONTROL_TIME_MINUTES_CUMULATIVE"])
cumulative_features3["f_SIGNIFICANT_STRIKES_GROUND_LANDED/ABSORBED_PER_MINUTE_OF_CONTROL_DIFF_CUMULATIVE"] = cumulative_features3["f_SIGNIFICANT_STRIKES_GROUND_LANDED_PER_MINUTE_OF_CONTROL_CUMULATIVE"] - cumulative_features3["f_SIGNIFICANT_STRIKES_GROUND_ABSORBED_PER_MINUTE_OF_CONTROL_CUMULATIVE"]

cumulative_features3["f_AVERAGE_CONTROL_TIME_MINUTES_PER_TAKEDOWN/REVERSAL/KNOCKDOWN_CUMULATIVE"] = cumulative_features3["f_CONTROL_TIME_MINUTES_CUMULATIVE"] / (1 + cumulative_features3["f_TAKEDOWNS_LANDED_CUMULATIVE"] + cumulative_features3["f_REVERSALS_CUMULATIVE"] + cumulative_features3["f_KNOCKDOWNS_CUMULATIVE"])
cumulative_features3["f_AVERAGE_CONTROL_TIME_MINUTES_GIVEN_UP_PER_TAKEDOWN/REVERSAL/KNOCKDOWN_CUMULATIVE"] = cumulative_features3["o_CONTROL_TIME_MINUTES_CUMULATIVE"] / (1 + cumulative_features3["o_TAKEDOWNS_LANDED_CUMULATIVE"] + cumulative_features3["o_REVERSALS_CUMULATIVE"] + cumulative_features3["o_KNOCKDOWNS_CUMULATIVE"])
cumulative_features3["f_AVERAGE_CONTROL_TIME_MINUTES_PER_TAKEDOWN/REVERSAL/KNOCKDOWN_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features3["f_AVERAGE_CONTROL_TIME_MINUTES_PER_TAKEDOWN/REVERSAL/KNOCKDOWN_CUMULATIVE"] - cumulative_features3["f_AVERAGE_CONTROL_TIME_MINUTES_GIVEN_UP_PER_TAKEDOWN/REVERSAL/KNOCKDOWN_CUMULATIVE"]

cumulative_features3["f_SIGNIFICANT_STRIKES_GROUND_LANDED_PER_MINUTE_OF_CONTROL_PER_TAKEDOWN/REVERSAL/KNOCKDOWN_CUMULATIVE"] = cumulative_features3["f_SIGNIFICANT_STRIKES_GROUND_LANDED_PER_MINUTE_OF_CONTROL_CUMULATIVE"] / (1 + cumulative_features3["f_TAKEDOWNS_LANDED_CUMULATIVE"] + cumulative_features3["f_REVERSALS_CUMULATIVE"] + cumulative_features3["f_KNOCKDOWNS_CUMULATIVE"])
cumulative_features3["f_SIGNIFICANT_STRIKES_GROUND_ABSORBED_PER_MINUTE_OF_CONTROL_PER_TAKEDOWN/REVERSAL/KNOCKDOWN_CUMULATIVE"] = cumulative_features3["f_SIGNIFICANT_STRIKES_GROUND_ABSORBED_PER_MINUTE_OF_CONTROL_CUMULATIVE"] / (1 + cumulative_features3["o_TAKEDOWNS_LANDED_CUMULATIVE"] + cumulative_features3["o_REVERSALS_CUMULATIVE"] + cumulative_features3["o_KNOCKDOWNS_CUMULATIVE"])
cumulative_features3["f_SIGNIFICANT_STRIKES_GROUND_LANDED/ABSORBED_PER_MINUTE_OF_CONTROL_PER_TAKEDOWN/REVERSAL/KNOCKDOWN_DIFF_CUMULATIVE"] = cumulative_features3["f_SIGNIFICANT_STRIKES_GROUND_LANDED_PER_MINUTE_OF_CONTROL_PER_TAKEDOWN/REVERSAL/KNOCKDOWN_CUMULATIVE"] - cumulative_features3["f_SIGNIFICANT_STRIKES_GROUND_ABSORBED_PER_MINUTE_OF_CONTROL_PER_TAKEDOWN/REVERSAL/KNOCKDOWN_CUMULATIVE"]

cumulative_features3["f_KO/TKO/KNOCKDOWNS_PER_SIGNIFICANT_STRIKE_LANDED_CUMULATIVE"] = (cumulative_features3["f_WINS_BY_KO/TKO_CUMULATIVE"] + cumulative_features3["f_KNOCKDOWNS_CUMULATIVE"]) / cumulative_features3["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features3["f_KO/TKO/KNOCKDOWNS_PER_SIGNIFICANT_STRIKE_ABSORBED_CUMULATIVE"] = (cumulative_features3["f_LOSSES_BY_KO/TKO_CUMULATIVE"] + cumulative_features3["o_KNOCKDOWNS_CUMULATIVE"]) / cumulative_features3["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features3["f_KO/TKO/KNOCKDOWNS_PER_SIGNIFICANT_STRIKE_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features3["f_KO/TKO/KNOCKDOWNS_PER_SIGNIFICANT_STRIKE_LANDED_CUMULATIVE"] - cumulative_features3["f_KO/TKO/KNOCKDOWNS_PER_SIGNIFICANT_STRIKE_ABSORBED_CUMULATIVE"]

cumulative_features3["f_FIGHTER_ID"] = feature_df["f_FIGHTER_ID"]
cumulative_features3["f_WIN"] = feature_df["f_WIN"]
cumulative_features3["f_LOSS"] = feature_df["f_LOSS"]
cumulative_features3["f_LOSS_BY_KO/TKO"] = feature_df["f_LOSS_BY_KO/TKO"]
cumulative_features3["f_WIN_TITLE_BOUT"] = feature_df["f_WIN_TITLE_BOUT"]
condition1 = cumulative_features3["f_WIN"].eq(0)
condition2 = cumulative_features3["f_LOSS"].eq(0)
condition3 = cumulative_features3["f_LOSS_BY_KO/TKO"].eq(0)
condition4 = cumulative_features3["f_WIN_TITLE_BOUT"].eq(0)
cumulative_features3["reset1"] = condition1.groupby(cumulative_features3.f_FIGHTER_ID).cumsum()
cumulative_features3["reset2"] = condition2.groupby(cumulative_features3.f_FIGHTER_ID).cumsum()
cumulative_features3["reset3"] = condition3.groupby(cumulative_features3.f_FIGHTER_ID).cumsum()
cumulative_features3["reset4"] = condition4.groupby(cumulative_features3.f_FIGHTER_ID).cumsum()
cumulative_features3["temp1"] = cumulative_features3["f_WIN"].mask(condition1, 0).groupby([cumulative_features3.f_FIGHTER_ID, cumulative_features3.reset1]).cumsum()
cumulative_features3["temp2"] = cumulative_features3["f_LOSS"].mask(condition2, 0).groupby([cumulative_features3.f_FIGHTER_ID, cumulative_features3.reset2]).cumsum()
cumulative_features3["temp3"] = cumulative_features3["f_LOSS_BY_KO/TKO"].mask(condition3, 0).groupby([cumulative_features3.f_FIGHTER_ID, cumulative_features3.reset3]).cumsum()
cumulative_features3["temp4"] = cumulative_features3["f_WIN_TITLE_BOUT"].mask(condition4, 0).groupby([cumulative_features3.f_FIGHTER_ID, cumulative_features3.reset4]).cumsum()
cumulative_features3["f_WIN_STREAK_CUMULATIVE"] = cumulative_features3.groupby("f_FIGHTER_ID")["temp1"].shift()
cumulative_features3["f_LOSS_STREAK_CUMULATIVE"] = cumulative_features3.groupby("f_FIGHTER_ID")["temp2"].shift()
cumulative_features3["f_LOSS_BY_KO/TKO_STREAK_CUMULATIVE"] = cumulative_features3.groupby("f_FIGHTER_ID")["temp3"].shift()
cumulative_features3["f_WIN_TITLE_BOUT_STREAK_CUMULATIVE"] = cumulative_features3.groupby("f_FIGHTER_ID")["temp4"].shift()
cumulative_features3 = cumulative_features3.drop(["f_FIGHTER_ID", "f_WIN", "f_LOSS", "f_LOSS_BY_KO/TKO", "f_WIN_TITLE_BOUT", "reset1", "reset2", "reset3", "reset4", "temp1", "temp2", "temp3", "temp4"], axis=1)

cumulative_features3["f_AGE_DAYS"] = (feature_df["DATE"] - feature_df["f_DATE_OF_BIRTH"]).dt.days
cumulative_features3["f_CTE_SCORE"] = cumulative_features3["f_AGE_DAYS"] * cumulative_features3["o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] * (1 + cumulative_features3["f_LOSSES_BY_KO/TKO_CUMULATIVE"])
# cumulative_features3["f_DAYS_SINCE_UFC_DEBUT"] = 
cumulative_features3["f_DAYS_SINCE_LAST_BOUT"] = feature_df.groupby("f_FIGHTER_ID")["DATE"].diff().dt.days
# cumulative_features3["f_DAYS_SINCE_LAST_LOSS_BY_KO/TKO"] = 
# cumulative_features3["f_DAYS_AS_CHAMPION"] = 

cumulative_features3

,f_TOTAL_TIME_MINUTES_CUMULATIVE,f_KNOCKDOWNS_CUMULATIVE,f_TOTAL_STRIKES_LANDED_CUMULATIVE,f_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE,f_TAKEDOWNS_LANDED_CUMULATIVE,f_TAKEDOWNS_ATTEMPTED_CUMULATIVE,f_SUBMISSION_ATTEMPTS_CUMULATIVE,f_REVERSALS_CUMULATIVE,f_CONTROL_TIME_MINUTES_CUMULATIVE,f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE,o_KNOCKDOWNS_CUMULATIVE,o_TOTAL_STRIKES_LANDED_CUMULATIVE,o_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE,o_TAKEDOWNS_LANDED_CUMULATIVE,o_TAKEDOWNS_ATTEMPTED_CUMULATIVE,o_SUBMISSION_ATTEMPTS_CUMULATIVE,o_REVERSALS_CUMULATIVE,o_CONTROL_TIME_MINUTES_CUMULATIVE,o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE,f_TITLE_BOUTS_FOUGHT,f_WINS_TITLE_BOUTS,f_FIVE_ROUND_BOUTS_FOUGHT,f_WINS_FIVE_ROUND_BOUTS,f_WINS_CUMULATIVE,f_LOSSES_CUMULATIVE,f_DRAWS_CUMULATIVE,f_NO_CONTESTS_CUMULATIVE,f_TOTAL_BOUTS_FOUGHT,f_WINS_BY_KO/TKO_CUMULATIVE,f_WINS_BY_SUBMISSION_CUMULATIVE,f_WINS_BY_DECISION_CUMULATIVE,f_WINS_FIRST_ROUND_FINISH_CUMULATIVE,f_LOSSES_BY_KO/TKO_CUMULATIVE,f_LOSSES_BY_SUBMISSION_CUMULATIVE,f_LOSSES_BY_DECISION_CUMULATIVE,f_KNOCKDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE,f_KNOCKDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE,f_KNOCKDOWNS_LANDED/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,f_KNOCKDOWNS_LANDED/ABSORBED_DIFF_CUMULATIVE,f_TOTAL_STRIKES_ACCURACY_PROPORTION_CUMULATIVE,f_TOTAL_STRIKES_DEFENSE_PROPORTION_CUMULATIVE,f_TOTAL_STRIKES_ACCURACY/DEFENSE_MULT_CUMULATIVE,f_TOTAL_STRIKES_LANDED_PER_MINUTE_CUMULATIVE,f_TOTAL_STRIKES_ABSORBED_PER_MINUTE_CUMULATIVE,f_TOTAL_STRIKES_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE,f_TOTAL_STRIKES_LANDED/ABSORBED_DIFF_CUMULATIVE,f_TAKEDOWNS_ACCURACY_PROPORTION_CUMULATIVE,f_TAKEDOWNS_DEFENSE_PROPORTION_CUMULATIVE,f_TAKEDOWNS_ACCURACY/DEFENSE_MULT_CUMULATIVE,f_TAKEDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE,f_TAKEDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE,f_TAKEDOWNS_LANDED/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,f_TAKEDOWNS_LANDED/ABSORBED_DIFF_CUMULATIVE,f_SUBMISSION_ATTEMPTS_ACCURACY_PROPORTION_CUMULATIVE,f_SUBMISSION_ATTEMPTS_DEFENSE_PROPORTION_CUMULATIVE,f_SUBMISSION_ATTEMPTS_ACCURACY/DEFENSE_MULT_CUMULATIVE,f_SUBMISSION_ATTEMPTS_PER_15_MINUTES_CUMULATIVE,f_SUBMISSION_ATTEMPTS_ABSORBED_PER_15_MINUTES_CUMULATIVE,f_SUBMISSION_ATTEMPTS/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,f_SUBMISSION_ATTEMPTS/ABSORBED_DIFF_CUMULATIVE,f_REVERSALS_ACCURACY_PROPORTION_CUMULATIVE,f_REVERSALS_DEFENSE_PROPORTION_CUMULATIVE,f_REVERSALS_ACCURACY/DEFENSE_MULT_CUMULATIVE,f_REVERSALS_PER_15_MINUTES_CUMULATIVE,f_REVERSALS_ABSORBED_PER_15_MINUTES_CUMULATIVE,f_REVERSALS/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,f_REVERSALS/ABSORBED_DIFF_CUMULATIVE,f_CONTROL_TIME_PROPORTION_CUMULATIVE,f_CONTROL_TIME_GIVEN_UP_PROPORTION_CUMULATIVE,f_CONTROL_TIME/CONTROL_TIME_GIVEN_UP_PROPORTION_DIFF_CUMULATIVE,f_CONTROL_TIME/CONTROL_TIME_GIVEN_UP_DIFF_CUMULATIVE,f_

In [7]:
class Elo:
    def __init__(self, k: int):
        self.rating_dict = {}
        self.k = k
    
    def __getitem__(self, player: str) -> float:
            return self.rating_dict[player]

    def __setitem__(self, player: str, data: float) -> None:
            self.rating_dict[player] = data

    def get_margin_factor(self, score: float) -> float:
            return np.log2(score + 1)
    
    def get_inflation_factor(self, r_win: float, r_lose: float) -> float:
        return 1 / (1 - ((r_lose - r_win) / 2200))
    
    def add_player(self, name: str, rating: float = 1500.):
            self.rating_dict[name] = rating
    
    def update_ratings(self, winner: str, loser: str, score: float) -> None:
        expected_result = self.get_expect_result(
            self.rating_dict[winner], self.rating_dict[loser]
        )
        margin_factor = self.get_margin_factor(score)
        inflation_factor = self.get_inflation_factor(
            self.rating_dict[winner], self.rating_dict[loser]
        )
        self.rating_dict[winner] = self.rating_dict[winner] + self.k * margin_factor * inflation_factor * (1 - expected_result)
        self.rating_dict[loser] = self.rating_dict[loser] + self.k * margin_factor * inflation_factor * (-1 + expected_result)

    def get_expect_result(self, p1: float, p2: float) -> float:
        exp = (p2 - p1) / 400.0
        return 1 / ((10.0 ** (exp)) + 1)

In [8]:
def get_win_method_weight(x: str) -> float:
    if "Decision" in x:
        if "Split" in x:
            return 1.
        else:
            return 3.
    elif "KO" in x:
        return 5.
    elif "Submission" in x:
        return 5.
    else:
        return 0.

In [9]:
elo_df = pd.DataFrame()
elo_df["BOUT_ID"] = df["BOUT_ID"]
elo_df["RED_FIGHTER_ID"] = df["RED_FIGHTER_ID"]
elo_df["BLUE_FIGHTER_ID"] = df["BLUE_FIGHTER_ID"]
elo_df["RED_OUTCOME"] = df["RED_OUTCOME"]
elo_df["OUTCOME_METHOD"] = df["OUTCOME_METHOD"]

def calc_elo_rank(df: pd.DataFrame) -> pd.DataFrame:
    elo_scorer = Elo(k = 20)
    fighter_set = set(df["RED_FIGHTER_ID"].unique().tolist() + df["BLUE_FIGHTER_ID"].unique().tolist())
    for fighter in fighter_set:
        elo_scorer.add_player(fighter)
    
    elo_red_fighter = np.zeros((len(df,)))
    elo_blue_fighter = np.zeros((len(df,)))

    for rowidx, fs_row in enumerate(df.itertuples()):
        red_fighter = fs_row.RED_FIGHTER_ID
        blue_fighter = fs_row.BLUE_FIGHTER_ID
        red_outcome = fs_row.RED_OUTCOME
        outcome_method = fs_row.OUTCOME_METHOD

        if red_outcome == "L":
            winner = blue_fighter
            loser = red_fighter
        else:
            winner = red_fighter
            loser = blue_fighter
        
        score = get_win_method_weight(outcome_method)

        elo_red_fighter[rowidx] = elo_scorer[red_fighter]
        elo_blue_fighter[rowidx] = elo_scorer[blue_fighter]

        elo_scorer.update_ratings(winner, loser, score)
    
    df["RED_FIGHTER_ELO"] = elo_red_fighter
    df["BLUE_FIGHTER_ELO"] = elo_blue_fighter

    return df

elo_df = calc_elo_rank(elo_df)
elo_df["RED_FIGHTER_ELO_PROBABILITY"] = 1 / (10 ** (-(elo_df["RED_FIGHTER_ELO"] - elo_df["BLUE_FIGHTER_ELO"]) / 400) + 1)
elo_df

,BOUT_ID,RED_FIGHTER_ID,BLUE_FIGHTER_ID,RED_OUTCOME,OUTCOME_METHOD,RED_FIGHTER_ELO,BLUE_FIGHTER_ELO,RED_FIGHTER_ELO_PROBABILITY
0,567a09fd200cfa05,279093302a6f44b3,96eff1a628adcc7f,W,KO/TKO,1500.000000,1500.000000,0.500000
1,2d2bbc86e941e05c,598a58db87b890ee,d3711d3784b76255,W,KO/TKO,1500.000000,1500.000000,0.500000
2,cecdc0da584274b9,429e7d3725852ce9,a5c53b3ddb31cc7d,W,Submission,1500.000000,1500.000000,0.500000
3,46acd54cc0c905fb,63b65af1c5cb02cb,46c8ec317aff28ac,W,Submission,1500.000000,1500.000000,0.500000
4,ac7ca2ec38b96c1a,279093302a6f44b3,598a58db87b890ee,W,KO/TKO,1525.849625,1525.849625,0.500000
...,...,...,...,...,...,...,...,...
7284,560a971c6ab8b854,009c4420727149ea,6aae253467e8f9ce,W,Decision - Unanimous,1460.680378,1463.881984,0.495393
7285,c2f730cb33df5221,9673a497a9da119a,abcf2cc6efc42031,L,Submission,1521.963684,1499.611156,0.532124
7286,c57a4810c932b296,749f572d1d3161fb,d0aaedcf7ccd0b45,W,KO/TKO,1542.021651,1536.306557,0.508224
7287,1b1fd32485861655,d247691a6c0e9034,2c9d00968f818270,W,Decision - Unanimous,1570.493701,1529.205283,0.559141


In [10]:
cumulative_features3["f_ELO"] = elo_df[["RED_FIGHTER_ELO", "BLUE_FIGHTER_ELO"]].stack().reset_index(drop=True).values
cumulative_features3["f_FIGHTER_ID"] = feature_df["f_FIGHTER_ID"]
cumulative_features3["f_ELO_CHANGE_SINCE_LAST_BOUT"] = cumulative_features3.groupby("f_FIGHTER_ID")["f_ELO"].diff()
cumulative_features3 = cumulative_features3.drop(columns=["f_FIGHTER_ID"])
cumulative_features3

,f_TOTAL_TIME_MINUTES_CUMULATIVE,f_KNOCKDOWNS_CUMULATIVE,f_TOTAL_STRIKES_LANDED_CUMULATIVE,f_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE,f_TAKEDOWNS_LANDED_CUMULATIVE,f_TAKEDOWNS_ATTEMPTED_CUMULATIVE,f_SUBMISSION_ATTEMPTS_CUMULATIVE,f_REVERSALS_CUMULATIVE,f_CONTROL_TIME_MINUTES_CUMULATIVE,f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE,o_KNOCKDOWNS_CUMULATIVE,o_TOTAL_STRIKES_LANDED_CUMULATIVE,o_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE,o_TAKEDOWNS_LANDED_CUMULATIVE,o_TAKEDOWNS_ATTEMPTED_CUMULATIVE,o_SUBMISSION_ATTEMPTS_CUMULATIVE,o_REVERSALS_CUMULATIVE,o_CONTROL_TIME_MINUTES_CUMULATIVE,o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE,f_TITLE_BOUTS_FOUGHT,f_WINS_TITLE_BOUTS,f_FIVE_ROUND_BOUTS_FOUGHT,f_WINS_FIVE_ROUND_BOUTS,f_WINS_CUMULATIVE,f_LOSSES_CUMULATIVE,f_DRAWS_CUMULATIVE,f_NO_CONTESTS_CUMULATIVE,f_TOTAL_BOUTS_FOUGHT,f_WINS_BY_KO/TKO_CUMULATIVE,f_WINS_BY_SUBMISSION_CUMULATIVE,f_WINS_BY_DECISION_CUMULATIVE,f_WINS_FIRST_ROUND_FINISH_CUMULATIVE,f_LOSSES_BY_KO/TKO_CUMULATIVE,f_LOSSES_BY_SUBMISSION_CUMULATIVE,f_LOSSES_BY_DECISION_CUMULATIVE,f_KNOCKDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE,f_KNOCKDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE,f_KNOCKDOWNS_LANDED/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,f_KNOCKDOWNS_LANDED/ABSORBED_DIFF_CUMULATIVE,f_TOTAL_STRIKES_ACCURACY_PROPORTION_CUMULATIVE,f_TOTAL_STRIKES_DEFENSE_PROPORTION_CUMULATIVE,f_TOTAL_STRIKES_ACCURACY/DEFENSE_MULT_CUMULATIVE,f_TOTAL_STRIKES_LANDED_PER_MINUTE_CUMULATIVE,f_TOTAL_STRIKES_ABSORBED_PER_MINUTE_CUMULATIVE,f_TOTAL_STRIKES_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE,f_TOTAL_STRIKES_LANDED/ABSORBED_DIFF_CUMULATIVE,f_TAKEDOWNS_ACCURACY_PROPORTION_CUMULATIVE,f_TAKEDOWNS_DEFENSE_PROPORTION_CUMULATIVE,f_TAKEDOWNS_ACCURACY/DEFENSE_MULT_CUMULATIVE,f_TAKEDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE,f_TAKEDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE,f_TAKEDOWNS_LANDED/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,f_TAKEDOWNS_LANDED/ABSORBED_DIFF_CUMULATIVE,f_SUBMISSION_ATTEMPTS_ACCURACY_PROPORTION_CUMULATIVE,f_SUBMISSION_ATTEMPTS_DEFENSE_PROPORTION_CUMULATIVE,f_SUBMISSION_ATTEMPTS_ACCURACY/DEFENSE_MULT_CUMULATIVE,f_SUBMISSION_ATTEMPTS_PER_15_MINUTES_CUMULATIVE,f_SUBMISSION_ATTEMPTS_ABSORBED_PER_15_MINUTES_CUMULATIVE,f_SUBMISSION_ATTEMPTS/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,f_SUBMISSION_ATTEMPTS/ABSORBED_DIFF_CUMULATIVE,f_REVERSALS_ACCURACY_PROPORTION_CUMULATIVE,f_REVERSALS_DEFENSE_PROPORTION_CUMULATIVE,f_REVERSALS_ACCURACY/DEFENSE_MULT_CUMULATIVE,f_REVERSALS_PER_15_MINUTES_CUMULATIVE,f_REVERSALS_ABSORBED_PER_15_MINUTES_CUMULATIVE,f_REVERSALS/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,f_REVERSALS/ABSORBED_DIFF_CUMULATIVE,f_CONTROL_TIME_PROPORTION_CUMULATIVE,f_CONTROL_TIME_GIVEN_UP_PROPORTION_CUMULATIVE,f_CONTROL_TIME/CONTROL_TIME_GIVEN_UP_PROPORTION_DIFF_CUMULATIVE,f_CONTROL_TIME/CONTROL_TIME_GIVEN_UP_DIFF_CUMULATIVE,f_

In [11]:
red_cumulative_features = cumulative_features3[::2]
blue_cumulative_features = cumulative_features3[1::2]
diffs_cumulative_features = red_cumulative_features - blue_cumulative_features

red_cumulative_features = red_cumulative_features.rename(columns=lambda x: "RED_" + x)
blue_cumulative_features = blue_cumulative_features.rename(columns=lambda x: "BLUE_" + x)
diffs_cumulative_features = diffs_cumulative_features.rename(columns=lambda x: "RED/BLUE_DIFF_" + x)

In [ ]:
# cumulative
# rolling 3 fight window
# lag 1 fight

# accuracy: f_landed / f_attempted
# avg landed per 15 min: f_landed / total_time * 15
# defense: (o_attempted - o_landed) / o_attempted
# avg absorbed per 15 min: o_landed / total_time * 15

# win pct (count draws as 0.5)
# winning and losing streaks
# finish pct
# wins by ko/tko, sub, dec

# avg landed per 15 min / avg absorbed per 15 min
# accuracy * defense
# total landed - total absorbed
# ground_strikes / control_time * 15
# (ground_strikes / control_time) / (takedowns_landed + reversals)
# control_time / total_time
# control_time / (takedowns_landed + reversals)
# (ko/tko_wins + knockdowns) / significant strikes landed
# head / total sig, body / total sig, leg / total sig, distance / total sig, clinch / total sig, ground / total sig
# number of days since last fight
# cte score: age * head_absorbed * (1 + ko/tko_losses)

# custom elo rating system
# elo score, elo probability

# mins and maxes
# deltas in stats between bouts

# title fight indicator
# champ indicator
# number of title fights
# number of 5-round fights participated in
# title wins

# finally finally ratios/differences between stats between fighters

# at the end, drop all number of attempted columns

### Coalesce the features into a single dataframe that will serve as input

In [100]:
data = pd.DataFrame()
data["DATE"] = pd.to_datetime(df["DATE"])
data["BOUT_WEIGHT_CLASS"] = df["BOUT_TYPE"].apply(lambda x: get_weight_class(x))
data["BOUT_IS_FEMALE"] = df["BOUT_TYPE"].apply(lambda x: is_female(x))
data["BOUT_IS_TITLE_BOUT"] = df["BOUT_TYPE"].apply(lambda x: is_title_bout(x))

data = pd.concat([data, red_cumulative_features, blue_cumulative_features, diffs_cumulative_features, elo_df["RED_FIGHTER_ELO_PROBABILITY"].to_frame()], axis=1)
data["TARGET"] = df["RED_OUTCOME"].apply(lambda x: 1 if x == "W" else 0 if x == "L" else np.nan)

data = data.loc[data["DATE"] >= pd.to_datetime("2012-01-01")]
data = data.loc[(data["RED_f_TOTAL_BOUTS_FOUGHT"].notnull()) & (data["BLUE_f_TOTAL_BOUTS_FOUGHT"].notnull()) & (data["TARGET"].notnull()) & (data["BOUT_IS_FEMALE"] == 0)]
data = data.drop(columns=["DATE", "BOUT_IS_FEMALE"])
indices = data.index
data

,BOUT_WEIGHT_CLASS,BOUT_IS_TITLE_BOUT,RED_f_TOTAL_TIME_MINUTES_CUMULATIVE,RED_f_KNOCKDOWNS_CUMULATIVE,RED_f_TOTAL_STRIKES_LANDED_CUMULATIVE,RED_f_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE,RED_f_TAKEDOWNS_LANDED_CUMULATIVE,RED_f_TAKEDOWNS_ATTEMPTED_CUMULATIVE,RED_f_SUBMISSION_ATTEMPTS_CUMULATIVE,RED_f_REVERSALS_CUMULATIVE,RED_f_CONTROL_TIME_MINUTES_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE,RED_f_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE,RED_o_KNOCKDOWNS_CUMULATIVE,RED_o_TOTAL_STRIKES_LANDED_CUMULATIVE,RED_o_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE,RED_o_TAKEDOWNS_LANDED_CUMULATIVE,RED_o_TAKEDOWNS_ATTEMPTED_CUMULATIVE,RED_o_SUBMISSION_ATTEMPTS_CUMULATIVE,RED_o_REVERSALS_CUMULATIVE,RED_o_CONTROL_TIME_MINUTES_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE,RED_o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE,RED_f_TITLE_BOUTS_FOUGHT,RED_f_WINS_TITLE_BOUTS,RED_f_FIVE_ROUND_BOUTS_FOUGHT,RED_f_WINS_FIVE_ROUND_BOUTS,RED_f_WINS_CUMULATIVE,RED_f_LOSSES_CUMULATIVE,RED_f_DRAWS_CUMULATIVE,RED_f_NO_CONTESTS_CUMULATIVE,RED_f_TOTAL_BOUTS_FOUGHT,RED_f_WINS_BY_KO/TKO_CUMULATIVE,RED_f_WINS_BY_SUBMISSION_CUMULATIVE,RED_f_WINS_BY_DECISION_CUMULATIVE,RED_f_WINS_FIRST_ROUND_FINISH_CUMULATIVE,RED_f_LOSSES_BY_KO/TKO_CUMULATIVE,RED_f_LOSSES_BY_SUBMISSION_CUMULATIVE,RED_f_LOSSES_BY_DECISION_CUMULATIVE,RED_f_KNOCKDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE,RED_f_KNOCKDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE,RED_f_KNOCKDOWNS_LANDED/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,RED_f_KNOCKDOWNS_LANDED/ABSORBED_DIFF_CUMULATIVE,RED_f_TOTAL_STRIKES_ACCURACY_PROPORTION_CUMULATIVE,RED_f_TOTAL_STRIKES_DEFENSE_PROPORTION_CUMULATIVE,RED_f_TOTAL_STRIKES_ACCURACY/DEFENSE_MULT_CUMULATIVE,RED_f_TOTAL_STRIKES_LANDED_PER_MINUTE_CUMULATIVE,RED_f_TOTAL_STRIKES_ABSORBED_PER_MINUTE_CUMULATIVE,RED_f_TOTAL_STRIKES_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE,RED_f_TOTAL_STRIKES_LANDED/ABSORBED_DIFF_CUMULATIVE,RED_f_TAKEDOWNS_ACCURACY_PROPORTION_CUMULATIVE,RED_f_TAKEDOWNS_DEFENSE_PROPORTION_CUMULATIVE,RED_f_TAKEDOWNS_ACCURACY/DEFENSE_MULT_CUMULATIVE,RED_f_TAKEDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE,RED_f_TAKEDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE,RED_f_TAKEDOWNS_LANDED/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,RED_f_TAKEDOWNS_LANDED/ABSORBED_DIFF_CUMULATIVE,RED_f_SUBMISSION_ATTEMPTS_ACCURACY_PROPORTION_CUMULATIVE,RED_f_SUBMISSION_ATTEMPTS_DEFENSE_PROPORTION_CUMULATIVE,RED_f_SUBMISSION_ATTEMPTS_ACCURACY/DEFENSE_MULT_CUMULATIVE,RED_f_SUBMISSION_ATTEMPTS_PER_15_MINUTES_CUMULATIVE,RED_f_SUBMISSION_ATTEMPTS_ABSORBED_PER_15_MINUTES_CUMULATIVE,RED_f_SUBMISSION_ATTEMPTS/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,RED_f_SUBMISSION_ATTEMPTS/ABSORBED_DIFF_CUMULATIVE,RED_f_REVERSALS_ACCURACY_PROPORTION_CUMULATIVE,RED_f_REVERSALS_DEFENSE_PROPORTION_CUMULATIVE,RED_f_REVERSALS_ACCURACY/DEFENSE